<h2>Stars and their spectra</h2>
<p><i>A python notebook studying stars in Gaia and SDSS. Written by Niall Deacon (MPIA), with some elements borrowed from a notebook by Rita Tojeiro (St Andrews).</i></p>
<p>Over the past few decades astronomy has seen an avalanche of data from large scale surveys. These surveys allow astronomers to do do large-scale statistical analysis on stars and galaxies, identify rare objects and help to choose targets for large observing facilities like the VLT or JWST.</p>
<p>In this lab we are going to use data from both the Gaia satellite and the Sloan Digital Sky Survey. Gaia measured the positions of billions stars extremely accurately over the course of several years. These measurements were then used to calculate the parallax (relating to how far away a star is) and proper motion (the apparent motion across the sky). The Sloan Digital Sky Survey (SDSS) is a series of ground-based surveys on relatively small telescopes in New Mexico and Chile. SDSS measured the spectra of four million stars, galaxies and exotic objects. Both Gaia and SDSS also measured the brighness of stars in different colours of light.</p>
<br/>
<p>We are going to be using the <a href="https://apps.sciserver.org/compute/">Compute environment</a> on the SDSS SciServer platform. You will need to register for an account on SciServer for this.</a></p>
<br/>
<br/>
<p>OK, let's get started. First we need to import some libraries.</p>

In [ ]:
import SciServer.CasJobs as CasJobs # query with CasJobs
import SciServer.SciDrive as SciDrive   # read/write to/from SciDrive
import SciServer.SkyServer as SkyServer   # show individual objects and generate thumbnail images through SkyServer
print('SciServer libraries imported')

In [ ]:
import matplotlib.pyplot as plt #used for plotting
import numpy as np
import astropy.io.fits as fits #used for reading FITS table data of SDSS spectra
print('Other libraries imported')

<p>Next we are going to query the SDSS database to extract some data. We do this by sending a request to the CASJobs server. </p>
<p>This job is written in Structured Query Language (SQL). SQL is used for a huge number of different databases, not just scientific ones, many large websites run using SQL for databases of products or articles.</p>
<p>There are a few different parts to this SQL query:</p>
<ul>
    <li><b>SELECT</b> which says what parameters you would like returned for each row in the database</li>
    <li><b>FROM</b> the name of the main (or in this case the only) table you are extracting data from</li>
    <li><b>WHERE</b> which restricts the results to only rows in the database matching certain criteria</li>
</ul>
<p>Here we are extracting an ID number for the object, the sky positions (Right Ascension - RA and Declination - Dec), four magnitudes (measures of apparent flux) measured in different colour filters and a couple of statistics relating to the quality of the photometry. We are restricting the search to one patch of the sky (see the restrictions in RA and Dec and to objects brighter than 19th magnitude in the g-band filter. Note we also select objects which have a high probability of being a point source. This means we will exclude fuzzy objects like galaxies. </p>

In [ ]:
query="""
SELECT p.objId,p.ra,p.dec,
    p.psfMag_u as u, p.psfMag_g as g, p.psfMag_r as r, p.psfMag_i as i, psfMag_z as z,p.clean,p.probPSF
FROM PhotoObjAll AS p
WHERE  p.probPSF>0.8 
  and p.ra between 180 and 190
  AND p.dec BETWEEN 10 AND 30
  and p.g < 19
"""
all_obj = CasJobs.executeQuery(query, "dr17")


<p>Now let's look at some of the rows in the dataframe we got back from the server</p>

In [ ]:
all_obj[0:10]

<p>Let's begin examining our data by plotting the sky positions of our objects to check the sky coverage of SDSS in this part of the sky</p>
<h3>Exercise</h3>
<p>Plot the sky positions (RA and Dec) of all the objects in the dataframe <code>all_obj</code>. Use marker '.' and size=1 as there are a lot of points.</p>

In [ ]:
#solution


<p>Did SDSS survey this entire area?</p>
<p>Small gaps in data like this are often caused by bright stars or extended objects like galaxies interfering with the measurements of other stars and galaxies causing them to either not be detected at all or to be detected with distorted parameters.  </p>

<p>Now let's take a look at the colours of the stars. In astronomy a colour is one magnitude subtracted from another. Typically the magnitude in a redder filter is subtracted from the magnitude in a bluer filter. In this system red objects have larger positive colours and bluer objects have smaller or even negative colours.</p>
<h3>Exercise</h3>
<p>Plot all the stars in the dataframe on a colour-colour plot. This is a scatter plot with one colour as the x-axis and another colour as the y-axis. In this case use the g-r colour for the y-axis with a range of -1 to 3 and the r-i colour for the x-axis with a range of -1 to 3.</p>

In [ ]:
#solution


<p>You should be able to see a distinct blob of points (more scientifically called a locus of points) where most of the stars appear. There are probably also a few odd lines jutting out. Can we improve on this diagram?</p>
<p>One thing we can do is plot only stars that have been flagged by the SDSS pipeline as having good photometry. To do this we make a slice of our data, selecting only objects with <code>clean=1</code> using the <code>np.where</code> function.</p>

In [ ]:
good_sdss_phot_slice=np.where(all_obj['clean']==1)[0]

<h3>Exercise</h3>
<p>Now remake your colour-colour plot to show only objects in the slice selected above.</p>

In [ ]:
#solution


# <p>Did all of the features from the previous colour-colour plot remain?</p>
<br/>
<p>It's worth noting that when calculating a colour we subtract one magnitude from another. This means that a colour contains information on how bright a star is in one band <b>in comparison</b> to another band. A colour doesn't contain information on how bright a star appears to the observer, or how bright it is in absolute terms.</p>

<h3>Exercise</h3>
<p>Make a colour-magnitude plot with r-i colour on the x-axis and r magnitude on the y axis. Make your plot so that the r magnitude is 10 at the top of the plot and 20 at the bottom of the plot. Use the same plot symbols as before and only plot the stars we previously selected as having good photometry.</p>

In [ ]:
#solution


<p>You should have a big locus of points. Note that SDSS photometry saturates at 14 in the both the r and i bands. This means most objects brighter than this will not pass our good photometry cut. 
<ul>    
    <li>Do you notice any other odd features near this limit?</li>
    <li>Do you think they are real or caused by saturation? Hint, think about both saturation in r and i.</li>
</ul>


<p>The plot above shows how bright these stars appear to an observer on Earth. But a dim star close to us might appear to have the same brightness as an intrinsically very bright star far away from us. To solve this we will need more measurements, and for that we turn to the Gaia Satellite.</p>

<h2>Adding Gaia data</h2>
<p>Gaia is a space mission that precisely measured the positions, distances and motions across the sky of over a billion stars in our Galaxy. We will use these distances to calculate how intrinsically bright stars are.</p>
<p>Luckily the SDSS CAS jobs server has Gaia DR2 preloaded and already crossmatched with the SDSS photometry table. We can use our previous query to build a new query.</p>
<p>Firstly this is a big query so we need to dump the output into a temporary table in the <a href="http://skyserver.sdss.org/CasJobs/mydb.aspx">MyDB</a> area of CASJobs. This is a small private database that all SciServer users get where they can write output tables from their SQL queries.</p>

<p>Let's look at the new elements of the query:</p>
<ul>
    <li>Firstly we are writing the output <b>INTO</b> our myDB table</li>
    <li>Next we are combining the SDSS photometry table by using <b>JOIN</b> commands to link it to two other tables. One is <code>GaiaDR2.sdssdr9_best_neighbour</code>, a simple table that says which Gaia ID number an object with a particular SDSS ID number is linked to. We then use this Gaia ID number to <b>JOIN</b> with the main Gaia data table <code>GaiaDR2.gaia_source</code>. By doing this type of join, an object must appear in all three of these tables to be returned by our query.</li>
    <li>We are also using the <b>LEFT JOIN</b> command to add in data from the SDSS spectra table (we will use this later in the exercise. A <b>LEFT JOIN</b> is different from a standard <b>JOIN</b> as it includes all objects from the first table in the join (in our case the SDSS photometry table). Objects with no corresponding entry in the SDSS spectroscopy table will have NULL values for the three parameters we pull from this table.</li>
    </ul>
<p>Next let's examine the new parameters we are extracting from Gaia</p>
<ul>
    <li>Along with the reference position (RA and Dec) of each object measures by Gaia we have two <a href="https://en.wikipedia.org/wiki/Proper_motion">proper motions</a> (one in RA and one in Dec) measured by Gaia along with their associated errors. These tell us how fast a star is moving across the sky and in which direction/</li>
    <li>Next we have the <a href="https://en.wikipedia.org/wiki/Parallax">trigonometric parallax</a> of each star. This is a measure of how close or far away a star is. The parallax is measured here in milliarcseconds (one thousandth of a sixtieth of a sixtieth of a degree. A star's distance here is given by <code>1000.0/parallax</code>.</li>
    <li>We then have Gaia photometry measured in magnitudes in three different filters G (a broadband filter, not the same as the SDSS g magnitude) and two narrower filters BP and RP. We also include the significance of the detection for each of these filters. This is the flux in each filter divided by the error on that flux. Detections with higher significances are generally much more reliable than low significance detections.</li>
    <li>Finally we have the plate number, date (here called MJD) and fiber ID. These three parameters combined will allow use to examine SDSS spectra later in the exercise.</li> 
    </ul>
<p>We select only objects with positive trignometric parallaxes (measurement errors can lead to objects with very small parallaxes having parallaxes less than zero measured even though a real negative parallax in physically impossible). We also only select objects with significances above certain threshholds in the Gaia photometry bands. Finally we restrict ourselves to the same sky area as before.</p>

In [ ]:
query="""
SELECT gs.source_id,
  g.ref_epoch as ref_epoch, g.ra as ra, g.ra_error as ra_error, g.dec as dec, g.dec_error as dec_error,
  g.parallax as parallax, g.parallax_error as parallax_error, g.pmra as pmra, g.pmra_error as pmra_error, 
  g.pmdec as pmdec, g.pmdec_error as pmdec_error,g.phot_g_mean_mag as gaia_g_mag,g.phot_g_mean_flux_over_error as gaia_g_significance,
  g.phot_bp_mean_mag as gaia_bp_mag,g.phot_bp_mean_flux_over_error as gaia_bp_significance,
  g.phot_rp_mean_mag as gaia_rp_mag,g.phot_rp_mean_flux_over_error as gaia_rp_significance,
  s.plate, s.mjd, s.fiberid,s.run2d
  INTO mydb.gaia_obj_tmp
  FROM DR17.PhotoObj AS p
  JOIN GaiaDR2.sdssdr9_best_neighbour AS gs ON CAST(gs.original_ext_source_id AS bigint) = p.objid
  JOIN GaiaDR2.gaia_source AS g ON gs.source_id=g.source_id
  LEFT JOIN SpecObj AS s ON s.bestobjid = p.objid
  
WHERE p.ra BETWEEN 180 AND 190
  AND p.dec BETWEEN 0 AND 20
  AND g.parallax > 0
  AND g.phot_bp_mean_flux_over_error > 20
  AND g.phot_bp_mean_flux_over_error > 20
  AND g.phot_g_mean_flux_over_error > 50
"""

jobid = CasJobs.submitJob(query, context="DR19")
print("Submited Job to CASJobs with ID:"+str(jobid))


<p>We now need to wait for the job to complete, this may take 5-10 minutes. We can run a function to keep polling the server until the job is complete.</p>

In [ ]:
CasJobs.waitForJob(jobid, verbose=False, pollTime=5)

<p>Let's check that our table is in MyDB.</p>

In [ ]:
CasJobs.getTables(context='MyDB')

<p>And extract all the columns from our temporary table.</p>

In [ ]:
query="""
SELECT *
FROM myDB.gaia_obj_tmp
"""
all_gaia_obj = CasJobs.executeQuery(query, "myDB")

<p>Now we can take a quick look at the dataframe we got back from CASJobs</p>

In [ ]:
all_gaia_obj[0:10]

<h3>Exercise</h3>
<p>Make a colour-magnitude plot like you did previously but this time use Gaia BP-RP as the x-axis and Gaia G as the y axis. Set the limits of the x-axis as -1 and 4.</p>

In [ ]:
#solution


<p>This isn't a great imporovement on what we had before, although it is noticable that the Gaia photometry saturates at brighter magnitudes than the SDSS photometry.</p>

<p>Let's start making use of Gaia parallax measurement. To do this we will calculate the absolute magnitude of all our stars in the Gaia G band. Absolute magnitude combines both the apparent magnitude and the distance so we are left with a quantity that depends only on a star's luminosity in that filter band.</p>

In [ ]:
all_gaia_obj['gaia_g_abs']=all_gaia_obj['gaia_g_mag']-5.0*np.log10(1000.0/all_gaia_obj['parallax'])+5

<h3>Exercise</h3>
<p>Now remake your colour-magnitude plot as a colour-absolute magnitude plot. You can keep the x-axis limits the same but use -5 to 20 as the limits of the y-axis.</p>

In [ ]:
#solution


<p>We can see that things are starting to change. The huge blob from before has gone. Do you notice any other interesting features here?</p>

<p>But we can improve this further. Previously in the SQL query we selected object with only good Gaia photometry, but other than excluding objects with negative parallaxes we didn't make a cut on the parallax quality. Let's do that now.</p>

<h3>Exercise</h3>
<p>Create a slice in <code>all_gaia_obj</code> called <code>good_parallax_slice</code>. Select only objects there the <code>parallax</code> is at least ten times greater than the <code>parallax_error</code>.</p>

In [ ]:
#solution


<h3>Exercise</h3>
<p>Now replot your colour-absolute magnitude diagram.</p>

In [ ]:
#solution


<p>The plot should now look much cleaner and you should be able to see a few different collections of stars.</p>
<p>What you've just plotted is known as a Herztprung-Russell Diagram. This is probably the most important plot in stellar astronomy.</p>
<p>The x-axis here is the colour which roughly scales with temperature. Bluer objects (on the left) have higher tempertures, redder objects (on the right) have lower temperatures.</p>
<p>The y-axis shows how luminous an object is with the objects with larger positive values being fainter than objects with smaller positive or even negative values. Luminousity depends most strongly on two factors: the temperature and the size of each star. </p>
<p>Stars are typically classified by spectral type and luminosity class. Spectral type follows the different features that appear in the spectrum of a star. Stars of the same spectral type have roughly the same temperature. Luminosity class divides stars of similar spectral types and temperatures into distinct categories based on their brightness. The luminosity of a star depends on its temperature and size. If we have two stars of the same temperature and colour (and hence of the same spectral type) but of a different luminosity then the brighter star will have a larger radius than the fainter star. So stars of a brighter luminosity class will be larger than stars of the same spectral type but a fainter luminosity class. </p> 
<ul>
<li>Most stars in the Galaxy lie on the "main sequence", where stars burn hydrogen in their cores. Where on the diagram you plotted do you think this main sequence lies?</li>
    <li>Looking at the diagram you plotted can you spot any brighter and/or fainter luminosity classes of stars?</li>
    <li>Look at the line of very faint objects fainter than $G_{abs}=10$ and bluer than $BP-RP=1.5$. How many magnitudes fainter than a main sequence star of the same colour are they? </li>
    <li>The absolute magnitude of a star is proportional to $10\log_{10}T + 5\log_{10}R$ where $T$ is the effective temperature and $R$ is the radius of the star. Using the magnitude difference you calculated in the previous answer, how much smaller are these faint blue objects than main sequence stars of the same temperature?</li>
    <ul>
        <li>To do this write down an equation for the magnitude of the star on the main sequence and another equation for the star on the very faint sequence.</li>
        <li>Remember that $\log_{10}x-\log{10}y=\log_{10}\frac{x}{y}$ and that if $a=\log_{10}b$ then $10^a=b$</li>
    </ul>
    <li>Let's think about the Solar System. The Sun has a radius that is about 10 times bigger than Jupiter's radius and about 100 times bigger than the Earth's. Using your answer from the previous questions, what object in the Solar System are these faint blue objects comparable in size to?</li>
    <li>Given that these faint blue object are as hot as stars, but have the size you calculated in the previous question, what type of objects do you think they are?</li>
    <li>Repeat the previous four questions but this time compare the bright red objects brighter than $G_{abs}=0$ and redder than $BP-RP=2.5$ with main sequence stars of the same colour.</li>
    <li>Note that in the Solar System the Sun's radius is about 200 times smaller than the distance between the Earth and the Sun</li>
</ul>

<h3>Start of optional section</h3>

<br/>
<p>There is also another factor that changes the temperature and brightness of a star, it's metal content. In astronomy metal means any element that isn't hydrogen or helium. Hence nitrogen, carbon, oxygen, magnesium and iron are all metals in an astronomical context.</p>
<p>Metals are created in the cores of stars and in supernova explosions. Each generation of stars enriching the galaxy with more metals. This means that the material new generations of stars form from gets more and more metal-rich over times. So younger stars are a lot more rich in metals than older stars. So if we want to look at stars of different metallicities we need to find stars of different ages.</p>
<p>It's pretty hard to measure the age of a star, but there is a shortcut we can take. The Milky Way is made up of several different populations of stars. In the area around the Sun, the two largest populations are disk stars and halo stars. Disk stars are younger than 10 billion years old and, like the Sun, they orbit around the galaxy in a flat disk. In the area around the Sun the disk stars are all moving, more or less, in the same direction and speed as the Sun as they go around the Galaxy. Some will be faster, some slower, some will move in slightly different directions, but they mostly follow the same orbital flow around the Galaxy. Halo stars are a much older, and hence metal-poor, population of stars. They are all older than 10 billion years old and orbit the Galaxy in randomly aligned orbits. As halo stars don't move with the general orbit of disk stars, which near the Sun is about 200 kilometres per second, then they all appear to have very high velocities relative to the Sun and to an observer on Earth.</p>
<p>Let's find out a bit about the velocities our stars are moving at by calculating the tangential velocity. This is the velocity they are moving at perpendicular to the line of sight we view them from. To do this we first calculate the total proper motion of each star by combining the proper motion in RA and Dec. Then we muliply the proper motion by a constant and by the distance.</p>

In [ ]:
all_gaia_obj['pm']=np.sqrt(all_gaia_obj['pmra']*all_gaia_obj['pmra']+all_gaia_obj['pmdec']*all_gaia_obj['pmdec'])
all_gaia_obj['v_tan']=all_gaia_obj['pm']*4.74/all_gaia_obj['parallax']

<h3>Exercise</h3>
<p>Plot a histogram of tangential velocities for the stars you previously selected in the <code>good_parallax_slice</code>. Make the y-axis of the histogram with a logarithmic scale.</p>

In [ ]:
#solution


<p>On your histogram where do you think the bulk of stars in the Galactic disk that are moving at low velocities relative to the Sun lie? Where do you think the halo stars moving at high velocity relative to the Sun lie? Hint, where does the histogram change shape?</p>

<h3>Exercise</h3>
<p>Select stars with both good parallaxes and a high tangential velocity. Use your previous histogram to decide where to make the cut on tangential velocity.</p>

In [ ]:
#solution


<h3>Exercise</h3>
<p>Plot your Herzprung-Russell diagram again. This time as well as plotting all stars in black, plot the high tangential velocity stars on top as circles with <code>size=40</code> and <code>color='goldenrod'</code></p>

In [ ]:
#solution


<p>Where do the high velocity, metal-poor halo stars lie in comparison to other stars? What can you say about the position of metal-poor stars on a Hertzprung-Russell diagram?</p>

<h3>End of optional section</h3>



<br/>
<h2>Introducing spectra</h2>
<p>Next we are going to use SDSS data to look at the spectra of stars across the Hetzprung Russell diagram (HR diagram).</p>
<p>We already have the information on which objects have SDSS spectra from our SQL query so let's select a slice for objects with SDSS spectra.</p>

In [ ]:
spectra_objects_slice=np.where((all_gaia_obj['fiberid']>0)&(all_gaia_obj['parallax']/all_gaia_obj['parallax_error'] > 10.0))[0]

<h3>Exercise</h3>
<p>Plot your HR diagram with all stars as small black dots and stars with spectra as green upward pointing triangles. You don't need to plot the high velocity stars like you did in the previous plot.</h3>

In [ ]:
#solution

<p>Now let's make several slices of the data. This will give us samples from three different parts of the HR diagram.</p>

In [ ]:
spectra_red_objects_slice=np.where((all_gaia_obj['fiberid']>0)\
                               &(all_gaia_obj['parallax']/all_gaia_obj['parallax_error'] > 10.0)\
                              &((all_gaia_obj['gaia_bp_mag']-all_gaia_obj['gaia_rp_mag'])>2.5))[0]

spectra_bright_blue_objects_slice=np.where((all_gaia_obj['fiberid']>0)\
                               &(all_gaia_obj['parallax']/all_gaia_obj['parallax_error'] > 10.0)\
                              &((all_gaia_obj['gaia_bp_mag']-all_gaia_obj['gaia_rp_mag'])<1.0)\
                               &(all_gaia_obj['gaia_g_abs']<7))[0]

spectra_faint_blue_objects_slice=np.where((all_gaia_obj['fiberid']>0)\
                               &(all_gaia_obj['parallax']/all_gaia_obj['parallax_error'] > 10.0)\
                               &(all_gaia_obj['gaia_g_abs']>(8.0+4.0*(all_gaia_obj['gaia_bp_mag']-all_gaia_obj['gaia_rp_mag']))))[0]


<h3>Exercise</h3>
<p>Remake the previous HR diagram, keeping the black dots and green triangles. This time add the red objects as red upward pointing triangles, the bright blue objects as blue upward pointing triangles and the faint blue objects as magenta upward pointing triangles.</p>

In [ ]:
#solution


<p>Now we are going to look at the spectra of objects. A spectrum tells you how much light aan object gives off at a particular wavelength. Spectra will have a general shape, normally related to how red or blue and object is and will also have absorption lines from atoms, ions and molecules in their atmosphere. Spectra normally have smaller wavelength (bluer light) on the left of the x-axis and larger wavelength (redder light) on the right of the x-axis. The y-axis normally represents the flux (light energy per unit time, per unit area, per unit wavelength) at a specific wavelength.</p>
<p>Below is a short function that takes a list of objects and plots spectra for up to nine of those objects. A spectrum is a plot that shows you how much energy a star gives out at a particular wavelength.</p>

In [ ]:
def plot_nine_spectra(object_list,line_list=None):
    plt.figure(figsize=(15, 15)) 
    if len(object_list)>=9:
        n_plots=9
    else:
        n_plots=len(object_list)
    ind = np.random.randint(0,len(object_list),n_plots)
    for count,obj in enumerate(ind):
        selected_obj=object_list.iloc[obj]
        if count>=9:
            break
        target_url='https://data.sdss.org/sas/dr17/env/SPECTRO_REDUX/'+selected_obj['run2d']+'/spectra/'
        if selected_obj['run2d']=='v5_13_2': 
            target_url=target_url+'lite/'
        
        target_url=target_url+str(int(selected_obj['plate'])).zfill(4)+'/spec-'+str(int(selected_obj['plate'])).zfill(4)\
        +'-'+str(int(selected_obj['mjd']))+'-'+str(int(selected_obj['fiberid'])).zfill(4)+'.fits'
        print('loading: ',target_url)
        fits_obj = fits.open(target_url)
        
        print('downloaded '+target_url)
        spectrum_data = fits_obj[1].data
        header = fits_obj[1].header
        flux = [x[0] for x in spectrum_data]
        wavelength= [10.0**x[1] for x in spectrum_data]
        plt.subplot(3,3,count+1)
        
        if line_list:
            for line in line_list:
                plt.plot([line,line],[0.0,1.2*max(flux)],color='red',linestyle='--')
        plt.plot(wavelength,flux)
        plt.ylabel('flux')
        plt.xlabel('wavelength (Angstroms)')
        plt.ylim((0.9*min(flux),1.1*max(flux)))
        plt.title('plate=`'+str(int(selected_obj['plate']))+' mjd='+str(int(selected_obj['mjd']))\
                  +' fiberid='+str(int(selected_obj['fiberid'])))
    plt.show()

<p>Execute the bit of code in the next cell to plot the spectra of some of the bright blue objects.</p>

In [ ]:
plot_nine_spectra(all_gaia_obj.loc[spectra_bright_blue_objects_slice])

<p>Have a look at the spectra you just made.</p>
<ul>
<li>What is the general shape of the spectra?</li>
<li>Can you spot any sharp features in the spectra?</li>
</ul>

<br/>
<p>Stars have lots of different chemical elements in them although they are mostly composed of hydrogen and helium. Atoms and ions in the atmosphere of a star are picky eaters. Light coming up from hotter parts of the star passes through the star's atmosphere and the atoms and ions in the star's atmosphere take little nibbles out of the light at specific wavelengths. These are known as absorption lines. Atoms and ions only take these nibbles at specific temperatures so two stars with an identical chemical composition but a different temperature (and hence different colour and spectral type) will have different patterns of absorption lines.</p>
<br/>
<p>Let's take a look at hydrogen lines. Hydrogen lines appear in the ultraviolet, in visible light and the infrared. In visible light, where SDSS spectra are taken, the main lines from hydrogen are called the Balmer lines. Let's define those in the code block below.</p>

In [ ]:
balmer_lines=[6562.79,4861.35,4340.472,4101.734]

<h3>Exercise</h3>
<p>Use the <code>line_list</code> keyword in <code>plot_nine_spectra</code> to add the locations of Balmer lines to your spectra of the bright blue objects</p>

In [ ]:
#solution

<ul>
    <li>Do the spectra of the bright blue stars show absorption lines from hydrogen?</li>
</ul>




<br/>
<p>Now let's look at the spectra of the faint blue objects</p>
<h3>Exercise</h3>
<p>Repeat the plot above, but use the faint blue objects slice we previously made. Keep the balmer lines in the plots.</p>


In [ ]:
#solution

<ul>
<li>Do the faint blue objects have absorption lines from hydrogen?</li>
    <li>Do you notice anything different about the spectra a the balmer lines?</li>
</ul>

<p>Next we turn to the red objects.</p>
<h3>Exercise</h3>
<p>Again run <code>plot_nine_spectra</code> but this time use the red objects selection we previously made</p>

In [ ]:
#solution

<ul>
    <li>What is the general shape of the spectra and how does it compare to the shape you saw in the selections of blue objects from earlier?</li>
    <li>Do these red objects have absorption lines at the hydrogen?</li>
    <li>Do you see anything else happening at any of the lines of the Balmer series?</li>
    <li>Do you see anything that migt be other absorption features? Do they look like little nibbles in the spectra? What do you think could be causing them?</li>
</ul>

<p>Congratulations, you've made it to the end of this notebook. The topics you met here: extracting data via an SQL query, cleaning that data up, plotting it and then looking at subsamples are all essential skills for modern data science. From the astronomy side of things you've encountered colours, magnitudes, the Hertzprung-Russell diagram, luminosity classes, metallicity and basic spectroscopy.</p>